In [5]:
# import logging
import os
import re
from pathlib import Path
from collections import Counter

import matplotlib.pyplot as plt
import numpy
import torch
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import yaml
import scanpy as sc
from einops import rearrange

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
from mushroom.data.visium import get_common_channels, adata_from_visium

In [6]:
section_config = yaml.safe_load(
    open('/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/mushroom_inputs.yaml')
)
section_config

[{'data': [{'dtype': 'he',
    'filepath': '/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s0_he.tif'},
   {'dtype': 'visium',
    'filepath': '/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s0_visium.h5ad'}],
  'id': 's0',
  'position': 0},
 {'data': [{'dtype': 'multiplex',
    'filepath': '/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s1_multiplex.ome.tiff'}],
  'id': 's1',
  'position': 1},
 {'data': [{'dtype': 'multiplex',
    'filepath': '/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s2_multiplex.ome.tiff'}],
  'id': 's2',
  'position': 11},
 {'data': [{'dtype': 'he',
    'filepath': '/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s3_he.tif'},
   {'dtype': 'visium',
    'filepath': '/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s3_visium.h5ad'}],
  'id': 's3',
  'position': 20},
 {'data': [{'dtype': 'multiplex',
    'filepath': '/

In [7]:
fps = [entry['filepath'] for item in section_config for entry in item['data'] if entry['dtype']=='visium']
fps

['/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s0_visium.h5ad',
 '/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s3_visium.h5ad']

In [ ]:
a = sc.read_visium('/data/estorrs/mushroom/data/examples/HT397B1_v1/')

In [131]:
from vit_pytorch import ViT
from einops.layers.torch import Rearrange
from vector_quantize_pytorch import VectorQuantize

In [132]:
dim = 100
n_clusters = 20
vq = VectorQuantize(dim, n_clusters)

In [135]:
x_pre_quant = torch.rand(32, 64, dim)
x_quant, indices, _ = vq(x_pre_quant)
x_quant.shape, indices.shape

(torch.Size([32, 64, 100]), torch.Size([32, 64]))

In [138]:
vq.codebook.shape

torch.Size([20, 100])

In [140]:
dists = torch.cdist(x_pre_quant, vq.codebook)
dists.shape

torch.Size([32, 64, 20])

In [143]:
torch.argmin(dists[0, 0])

tensor(15)

In [144]:
indices[0, 0]

tensor(15)

In [153]:
indices

tensor([[15, 15, 17,  ..., 14,  0, 16],
        [17, 17,  1,  ...,  4, 15, 15],
        [ 4, 15,  0,  ..., 14, 16,  4],
        ...,
        [16, 17,  0,  ...,  0, 16,  4],
        [15, 16, 15,  ..., 15, 17, 16],
        [ 4,  3, 15,  ..., 17, 15, 15]])

In [147]:
x = torch.randint(0, 10, (32, 64))
y = torch.randint(0, 10, (32, 64))
x.shape, y.shape

(torch.Size([32, 64]), torch.Size([32, 64]))

In [150]:
torch.nn.functional.cross_entropy(torch.nn.functional.one_hot(x), y)

RuntimeError: "softmax_kernel_impl" not implemented for 'Long'